In [1]:
import pandas as pd
from pandas_gbq import to_gbq
from sqlalchemy import create_engine
from google.oauth2 import service_account
import dotenv

In [2]:
def load_database(db_name:str):
    # 데이터베이스 연결
    engine_name = "mysql+pymysql"
    user = 'root'
    password = 'sat_dreams12'
    host = 'localhost'
    port = 3306
    database = db_name

    return create_engine(f"{engine_name}://{user}:{password}@{host}:{port}/{database}")


In [3]:
# hackle 연결정보 등록
votes_mysql_conn = load_database('votes')

votes_mysql_conn.connect()

In [4]:
# 해당 데이터베이스의 테이븖 목록 조회
query = """
SHOW TABLES;
"""

# 쿼리문 조회
VOTES_TABLES = pd.read_sql(query, votes_mysql_conn)

# 테이블 목록 출력
print("Hackle Database Tables:")
for table in VOTES_TABLES.values:
    print(table[0])

Hackle Database Tables:
accounts_attendance
accounts_blockrecord
accounts_failpaymenthistory
accounts_friendrequest
accounts_group
accounts_nearbyschool
accounts_paymenthistory
accounts_pointhistory
accounts_school
accounts_timelinereport
accounts_user
accounts_user_contacts
accounts_userquestionrecord
accounts_userwithdraw
event_receipts
events
polls_question
polls_questionpiece
polls_questionreport
polls_questionset
polls_usercandidate


In [ ]:
# 각 테이블의 데이터를 DataFrame으로 읽어오기
for i, table_name in enumerate(VOTES_TABLES.values):
    query = f"SELECT * FROM votes.{table_name[0]}"
    df = pd.read_sql(query, votes_mysql_conn)

    # BigQuery에 업로드하기 위해 인증정보 가져오기
    dotenv.load_dotenv()
    key_path = dotenv.get_key(dotenv.find_dotenv(), "GOOGLE_APPLICATION_CREDENTIALS")

    credentials = service_account.Credentials.from_service_account_file(key_path)

    to_gbq(
        dataframe=df,
        destination_table=f'votes.{VOTES_TABLES.values[i][0]}',
        project_id=dotenv.get_key(dotenv.find_dotenv(), "GOOGLE_PROJECT_ID"),  # 여기에 실제 프로젝트 ID를 입력하세요
        if_exists='replace',  # 기존 테이블이 있으면 대체
        progress_bar=True,  # 진행 상황 표시
        credentials=credentials  
    )

100%|██████████| 1/1 [00:00<00:00, 11397.57it/s]
